In [1]:
from ast import literal_eval
from pprint import pprint as pp
import collections
import multiprocessing
import sys
import warnings

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import dask.dataframe as dd

%matplotlib inline
pd.set_option('max_columns', 999)
warnings.filterwarnings('ignore')

N_CPUS = multiprocessing.cpu_count()

In [2]:
train_df = pd.read_csv('data/train.csv')
print(train_df.shape)

train_df.head()

(11341042, 11)


,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


Example of heterogeneity of the `event_data` column.

In [3]:
train_df.head(5).event_data.apply(literal_eval).tolist()

[{'event_code': 2000, 'event_count': 1},
 {'event_code': 2000, 'event_count': 1},
 {'version': '1.0', 'event_count': 1, 'game_time': 0, 'event_code': 2000},
 {'description': "Let's build a sandcastle! First, fill up your mold with sand! You can use the shovel here. The mold gives the sand its shape!",
  'identifier': 'Dot_LetsSandcastle,Dot_FillMold,Dot_MoldShape',
  'media_type': 'audio',
  'total_duration': 6758,
  'event_count': 2,
  'game_time': 53,
  'event_code': 3010},
 {'description': "Let's build a sandcastle! First, fill up your mold with sand! You can use the shovel here. The mold gives the sand its shape!",
  'identifier': 'Dot_LetsSandcastle,Dot_FillMold,Dot_MoldShape',
  'media_type': 'audio',
  'duration': 6919,
  'event_count': 3,
  'game_time': 6972,
  'event_code': 3110}]

In [5]:
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def process_event_data(event_data):
    # relies on literal_eval, need str
    assert isinstance(event_data, str) 
    try:
        event_data = flatten_dict(literal_eval(event_data))
        event_data['bad_string'] = False
    except ValueError:
        event_data = dict(bad_string=True)
    return event_data

def organize_events_data(df):
    df = dd.from_pandas(df, npartitions=N_CPUS)
    events_list = df.event_data.apply(process_event_data).compute().tolist()
    events_df = pd.DataFrame({k: v for k,v in enumerate(events_list)}).T
    return events_df


%time train_events_df = organize_events_data(train_df.sample(1000000))

print(train_events_df.shape)
rejects = train_events_df.loc[train_events_df.bad_string==True]
print(rejects.shape)
print(rejects.shape[0] / train_events_df.shape[0])

del rejects
sys.stdout.flush()

train_events_df.head()

# _ = pd.DataFrame({k: v for k,v in enumerate(train_df.head(250).event_data.apply(process_event_data).tolist())}).T
# _.loc[_.bad_string==True]

CPU times: user 2min 14s, sys: 4.69 s, total: 2min 19s
Wall time: 2min 18s
(1000000, 110)
(161408, 110)
0.161408


,animals,bad_string,bottle_amount,bottle_color,bottles,bucket,buckets,buckets_placed,bug,bug_length,buglength,castles_placed,caterpillar,caterpillars,chests,cloud,cloud_size,container_type,coordinates_stage_height,coordinates_stage_width,coordinates_x,coordinates_y,crystal_id,crystals,current_containers,description,destination,diet,dinosaur,dinosaur_count,dinosaurs,dinosaurs_placed,distance,duration,dwell_time,end_position,event_code,event_count,exit_type,flower,flowers,game_time,gate_column,gate_row,gate_side,growth,hat,hats,height,holes,houses,identifier,jar,layout_row1,layout_row2,left,level,location,max_position,media_type,misses,mode,molds,movie_id,nest,object,object_type,options,pillars,position,previous_jars,prompt,resources,right,rocket,round,round_number,round_prompt,round_target_animal,round_target_size,round_target_type,scale_contents,scale_weights,session_duration,shell_size,shells,side,size,source,stage_number,starting_weights,stumps,table_weights,tape_length,target_bucket,target_containers,target_distances,target_size,target_water_level,target_weight,time_played,total_containers,total_duration,toy,toy_earned,tutorial_step,version,water_level,weight,weights
0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Drag the shovel to the molds to fill them up!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2100,NaN,NaN,3110,22,NaN,NaN,NaN,17056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dot_DragShovel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,762,1015,279,614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4070,28,NaN,NaN,NaN,18573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,762,1015,146,217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4070,50,NaN,NaN,NaN,33074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,762,1015,782,207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4030,99,NaN,NaN,NaN,48458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,762,1015,782,207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117,NaN,NaN,4035,100,NaN,NaN,NaN,48575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
